In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
trips_filename = '../data/201508_trip_data.csv'
stations_filename = '../data/201508_station_data.csv'

trips_pdf = pd.read_csv(trips_filename)
stations_pdf = pd.read_csv(stations_filename)

In [ ]:
display(stations_pdf.head())

In [ ]:
# create nodes from stations.
#
# although networkx used hashed values for nodes, i.e. we don't need to
# have 'id' column, we decided to keep the same format as GraphFrame (PySpark).
#
stations_pdf = stations_pdf.rename(columns={'station_id':'id'})

In [ ]:
# index on the station id, now we call it 'id'
stations_pdf.set_index('id', inplace = True)
# display(stations_pdf.head())

In [ ]:
# %%time
nodes_dict = stations_pdf.to_dict('index')
nodes_list = [x for x in nodes_dict.items()]

In [ ]:
#display(nodes_list)

In [ ]:
display(trips_pdf.tail())

In [ ]:
# pick our set of columns to represent the edges
edge_columns = ['Start Terminal', 'End Terminal', 'Trip ID', 'Start Date', 'End Date']

trips_edges_pdf = trips_pdf.loc[:,edge_columns]

# rename the start and end terminal to 'src' and 'dst'
# Although not reqired we'll use the sample column names as the ones used with DataFrame 
# (PySpark) "src" (source vertex ID of edge) and "dst" (destination vertex ID of edge).

trips_edges_pdf = trips_edges_pdf.rename(columns={'Start Terminal':'src', 'End Terminal':'dst'})
trips_edges_pdf.shape

In [ ]:
# display(trips_edges_pdf.head())

In [ ]:
# trips_edges_pdf.loc[0, 'src']

In [ ]:
# %%time
edges_list = []
for i in range(trips_edges_pdf.shape[0]):
    edges_list.append((trips_edges_pdf.loc[i, 'src'], trips_edges_pdf.loc[i, 'dst'], 
                       {'Start Date': trips_edges_pdf.loc[i, 'Start Date'], 
                        'End Date': trips_edges_pdf.loc[i, 'End Date']}))

In [ ]:
# edges_list[0]

In [ ]:
%%time
g = nx.MultiDiGraph()

g.add_nodes_from(nodes_list)

g.add_edges_from(edges_list)

In [ ]:
%%time
g = nx.MultiDiGraph()
g.add_nodes_from(nodes_list)
for row in trips_edges_pdf.itertuples():
    edge = [(row.src, row.dst, 
                       {'Start Date': row._4, 
                        'End Date': row._5})]
    print(row[3])
    break
    g.add_edges_from(edge)

In [ ]:
trips_edges_pdf.iloc[1,2:]

In [ ]:
g.nodes[3]

In [ ]:
type(g.node)

In [ ]:
# g.get_edge_data(3,4, default=0)

In [ ]:
# g.edges(data=True)[0:5]

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
# plt.subplot(121)
# nx.draw(g, with_labels=True, font_weight='bold')
# plt.show()

In [ ]:
x = g.in_degree()

sorted(x, key=lambda x:x[1], reverse=True)

In [ ]:
print('node {} degrees - in:{}, out;{}'.format(70, g.in_degree(70), g.out_degree(70)))

In [ ]:
# from networkx.algorithms import community

# three_nodes_gen = community.k_clique_communities(g, 3)

# group = next(three_nodes_gen)
# count = 0
# while group:
#     group = next(three_nodes_gen)
#     count += 1

In [ ]:
# attempt in finding motif of three nodes.
# import itertools

# # find 3 nodes subgraphs
# target = nx.MultiDiGraph()
# target.add_edge(1,2)
# target.add_edge(2,3)

# count = 0
# for sub_nodes in itertools.combinations(g.nodes(),len(target.nodes())):
#     subg = g.subgraph(sub_nodes)
#     if nx.is_weakly_connected(subg) and nx.is_isomorphic(subg, target):
#         print(subg.edges())
#         count += 1

In [ ]:
df_adjacency = nx.to_pandas_adjacency(g)
df_edgelist = nx.to_pandas_edgelist(g)

In [ ]:
display(df_adjacency.head())

In [ ]:
display(df_edgelist.head())

In [ ]:
df_edgelist.shape

In [ ]:
centrality = nx.centrality.betweenness_centrality(g)
sorted(centrality.items(), key=lambda kv: kv[1], reverse=True)